In [16]:
#import the needed libraries
import numpy as np 
import pandas as pd 
import keras 
from keras import Sequential 
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten
from keras.layers import BatchNormalization
from keras import backend as K
from scipy.ndimage.interpolation import shift

In [17]:
#load the datasets from MNIST. I have divided the dataset into the training and test data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [18]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [19]:
#we need to preprocess the data; the train data dimension is initially (60000, 28, 28)
#the model requires one more dimension, and so I reshaped the matrix to be (600000, 28, 28, 1)
X_train = X_train.reshape (X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape (X_test.shape[0], 28, 28, 1)

In [20]:
#defining the shape of the input data
input_ = (28, 28, 1)

# Possible improvement method: Augmentation. Did not implement due to dataset size

In [21]:
# #image shifting function 
# def shift_image(img, ix, iy):
#     img = img.reshape((28, 28))
#     shifted_img = shift(img, [iy, ix], cval=0, mode="constant")
#     return shifted_img.reshape([-1])

In [22]:
# Xtrain_aug = [image for image in X_train]
# ytrain_aug = [image for image in y_train]

In [23]:
# for ix, iy in ((1,0), (-1,0), (0,1), (0,-1)):
#      for img, label in zip(X_train, y_train):
#              Xtrain_aug.append(shift_image(img, ix, iy))
#              ytrain_aug.append(label)

In [24]:
# shuffle_ind = np.random.permutation(len(Xtrain_aug))
# Xtrain_aug = np.array(Xtrain_aug)[shuffle_ind]
# ytrain_aug = np.array(ytrain_aug)[shuffle_ind]

In [25]:
#convert to binary class matrices
num_classes =  10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [26]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [27]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [28]:
#tuning the model hyperparameters to increase the model accuracy. note: another potential methods to do this could be augmentation
#but this is fine to prevent the model from overfitting due to the dataset size
# adding Batch Normalization to help stabilize the training process and improve the learning rate
batch_size = 128
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_uniform',input_shape=input_))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9),metrics=['accuracy'])

In [29]:
#fit and train the model
hist = model.fit(X_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, y_test))
print("The model has successfully trained")

Epoch 1/10
469/469 [==============================] - 291s 620ms/step - loss: 0.2310 - accuracy: 0.9341 - val_loss: 0.1225 - val_accuracy: 0.9602
Epoch 2/10
469/469 [==============================] - 383s 816ms/step - loss: 0.0897 - accuracy: 0.9723 - val_loss: 0.0427 - val_accuracy: 0.9853
Epoch 3/10
469/469 [==============================] - 377s 805ms/step - loss: 0.0636 - accuracy: 0.9799 - val_loss: 0.0387 - val_accuracy: 0.9882
Epoch 4/10
469/469 [==============================] - 340s 725ms/step - loss: 0.0499 - accuracy: 0.9839 - val_loss: 0.0350 - val_accuracy: 0.9890
Epoch 5/10
469/469 [==============================] - 321s 685ms/step - loss: 0.0454 - accuracy: 0.9853 - val_loss: 0.0316 - val_accuracy: 0.9898
Epoch 6/10
469/469 [==============================] - 344s 734ms/step - loss: 0.0399 - accuracy: 0.9871 - val_loss: 0.0408 - val_accuracy: 0.9876
Epoch 7/10
469/469 [==============================] - 449s 958ms/step - loss: 0.0339 - accuracy: 0.9885 - val_loss: 0.0319 -

In [30]:
#computing model loss value and accuracy on test data
accVal = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', accVal[0])
print('Test accuracy %:', accVal[1]*100)

Test loss: 0.03352177515625954
Test accuracy %: 99.04000163078308


In [31]:
#Result of tuning model structure: model accuracy change of 0.05, from initial accuracy
#The model was possibly overfitting, as deteced in the evaluation using the GUI
#saving the model
model.save('mnist.h5')

# GUI Application for model evaluation 

In [86]:
'''This is the GUI application to test/evaluate the model'''
'''To run it, save the model on your device and save this code with a .py extensiton, and run it'''
'''I will be deploying the GUI app and packaging it for presentation'''

from tkinter import *
from keras.models import load_model
import tkinter as tk
import numpy as np
import win32gui
from PIL import ImageGrab, Image

model = load_model('mnist.h5')

def predict_digit(img):
    #changing image size to 28 x 28 
    img = img.resize((28,28))
    img = img.convert('L')
    img = np.array(img)
    #preprocessing image to reshape to (1, 28,28, 1)
    img = img.reshape(1,28,28,1)
    img = img.astype('float32')
    img /= 255

    #predicting the model
    result = model.predict([img])[0]
    return np.argmax(result), max(result)

# the GUI app class
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        self.canvas = tk.Canvas(self, width=300, height=300, bg = "black", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 6
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='white')

app = App()
app.title("Draw Digit for the app to predict: ")
mainloop()

KeyboardInterrupt: 